In [1]:
import numpy as np
import scipy
from scipy.sparse import diags
from scipy.sparse.linalg import cg
from sys import getsizeof
from numpy.linalg import multi_dot as md
from tqdm.notebook import tqdm

In [2]:
%load_ext line_profiler

In [3]:
class ImplicitNaiveALS(object):
    
    def __init__(self, factors=10, iterations=100, lambda_=4, alpha=40, seed=10, cg=False):
        self.iterations = iterations
        self.lambda_ = lambda_
        self.alpha = alpha
        self.factors = factors
        self.Yi = None
        self.Xu = None
        self.seed = seed
        self.cg = cg
    
    def fit(self, matrix):
        np.random.seed(self.seed)
        Pui = (matrix > 0).astype('int')
        Cui = 1 + self.alpha * matrix
        Xu = 0.02 * np.random.normal(0, size=(matrix.shape[0], self.factors))
        Yi = 0.02 * np.random.normal(0, size=(matrix.shape[1], self.factors))
        
        for _iter in tqdm(range(self.iterations)):
            
            for user in range(matrix.shape[0]):
                a = md([Yi.T, np.diag(Cui[user]), Yi]) + self.lambda_ * np.eye(self.factors)
                b = md([Yi.T, np.diag(Cui[user]), Pui[user]])
                Xu[user] = self._solver(a, b)
            
            for item in range(matrix.shape[1]):
                a = md([Xu.T, np.diag(Cui[:, item]), Xu]) + self.lambda_ * np.eye(self.factors)
                b = md([Xu.T, np.diag(Cui[:, item]), Pui[:, item]])
                Yi[item] = self._solver(a, b)
                
        self.Xu = Xu
        self.Yi = Yi        
    
    
    def predict(self, user_id, top=10):
        recommendations = np.dot(self.Xu[user_id], self.Yi.T).argsort(axis=1)[:, -top:][:, ::-1]
        return recommendations
    
    def _solver(self, a, b):
        if self.cg == True:
            return scipy.sparse.linalg.cgs(a, b, maxiter=3)[0]
        else:
            return np.linalg.solve(a, b)

In [4]:
a = ImplicitNaiveALS(factors=100, iterations=10, cg=True)

In [5]:
size = 100
temp = 10 * np.eye(size) + 0.0001 * np.random.normal(0, size=(size,size))

In [7]:
a = ImplicitNaiveALS(factors=50, iterations=100, cg=False)
a.fit(temp)

In [8]:
a.predict(user_id=[1,2,3])

array([[ 1, 67, 10, 87, 20,  8, 22, 78, 42, 70],
       [ 2, 53, 70, 44, 56, 12, 17, 30, 24, 38],
       [ 3, 74, 20, 59, 29, 39, 75, 47, 72, 16]])